In [1]:
!pip install -q langgraph groq PyPDF2 python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.6 MB/s eta 0:00:00


In [2]:
import os
import json
from groq import Groq
from google.colab import userdata

# Add your Groq API key in Colab: Runtime → Manage secrets → GROQ_API_KEY
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
client = Groq(api_key=os.environ["GROQ_API_KEY"])

def call_llm(prompt: str) -> str:
    response = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="llama-3.1-8b-instant",  # ✅ Updated model
        temperature=0.0
    )
    return response.choices[0].message.content

In [3]:
import PyPDF2

def parse_resume(file_path: str) -> str:
    try:
        with open(file_path, "rb") as f:
            reader = PyDF2.PdfReader(f)
            return "\n".join(page.extract_text() or "" for page in reader.pages)
    except Exception as e:
        return f"ERROR_PARSING_RESUME: {str(e)}"

In [4]:
from typing import TypedDict, Dict, Any

class State(TypedDict):
    resume_text: str
    job_description: str
    candidate_profile: Dict[str, Any]
    job_requirements: Dict[str, Any]
    match_score: float
    confidence: float
    recommendation: str
    requires_human: bool
    reasoning_summary: str
    parsing_error: bool

In [5]:
def resume_parser_node(state: State) -> dict:
    resume_text = state["resume_text"]
    if "ERROR_PARSING_RESUME" in resume_text:
        return {
            "candidate_profile": {},
            "parsing_error": True
        }

    prompt = f"""
Extract structured info from this resume. Return ONLY JSON:
{{"name": "...", "years_experience": 0, "skills": ["..."], "education": "...", "domain_experience": ["..."]}}

Resume:
{resume_text}
"""
    try:
        text = call_llm(prompt)
        start = text.find("{")
        end = text.rfind("}")
        if start != -1 and end != -1:
            text = text[start:end+1]
        text = text.replace("\n", "").replace("\\", "").strip()
        profile = json.loads(text)
        return {"candidate_profile": profile, "parsing_error": False}
    except Exception:
        return {
            "candidate_profile": {
                "name": "Unknown",
                "years_experience": 0,
                "skills": [],
                "education": "Not extracted",
                "domain_experience": []
            },
            "parsing_error": False
        }

In [6]:
def jd_parser_node(state: State) -> dict:
    jd_text = state["job_description"]
    prompt = f"""
Extract job requirements. Return ONLY JSON:
{{"role": "...", "required_skills": ["..."], "required_experience_years": 0, "strictness_level": "strict"|"flexible"|"vague"}}

Job Description:
{jd_text}
"""
    try:
        text = call_llm(prompt)
        start = text.find("{")
        end = text.rfind("}")
        if start != -1 and end != -1:
            text = text[start:end+1]
        text = text.replace("\n", "").replace("\\", "").strip()
        reqs = json.loads(text)
        return {"job_requirements": reqs}
    except Exception:
        return {
            "job_requirements": {
                "role": "Unknown",
                "required_skills": [],
                "required_experience_years": 0,
                "strictness_level": "vague"
            }
        }

In [7]:
def match_node(state: State) -> dict:
    profile = state["candidate_profile"]
    job = state["job_requirements"]

    cand_skills = set(s.lower() for s in profile.get("skills", []))
    req_skills = set(s.lower() for s in job.get("required_skills", []))
    matched = cand_skills & req_skills
    skill_score = len(matched) / max(1, len(req_skills))

    cand_exp = profile.get("years_experience", 0)
    req_exp = job.get("required_experience_years", 0)
    exp_score = min(cand_exp / req_exp, 1.0) if req_exp > 0 else 0.7

    match_score = round(0.6 * skill_score + 0.4 * exp_score, 2)
    strictness = job.get("strictness_level", "flexible")

    if strictness == "vague":
        requires_human, recommendation, confidence = True, "Needs manual review", 0.7
    elif strictness == "strict":
        if match_score >= 0.85:
            requires_human, recommendation, confidence = False, "Proceed to interview", 0.95
        elif match_score >= 0.3:
            requires_human, recommendation, confidence = True, "Needs manual review", 0.75
        else:
            requires_human, recommendation, confidence = False, "Reject", 0.90
    else:  # flexible
        if match_score >= 0.75:
            requires_human, recommendation, confidence = False, "Proceed to interview", 0.90
        elif match_score >= 0.5:
            requires_human, recommendation, confidence = True, "Needs manual review", 0.70
        else:
            requires_human, recommendation, confidence = False, "Reject", 0.85

    reasoning = (
        f"Match score {match_score}. Matched skills: {list(matched)}. "
        f"Experience: {cand_exp}y vs {req_exp}y required. Strictness: {strictness}."
    )

    return {
        "match_score": match_score,
        "confidence": confidence,
        "recommendation": recommendation,
        "requires_human": requires_human,
        "reasoning_summary": reasoning
    }

In [8]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(State)

# Add nodes
workflow.add_node("parse_resume", resume_parser_node)
workflow.add_node("parse_jd", jd_parser_node)
workflow.add_node("match", match_node)

# Set entry point
workflow.set_entry_point("parse_resume")

# Define edges
workflow.add_edge("parse_resume", "parse_jd")
workflow.add_edge("parse_jd", "match")
workflow.add_edge("match", END)

app = workflow.compile()

In [9]:
from google.colab import files

# Upload files
print("📤 Upload resume (PDF):")
resume_file = list(files.upload().keys())[0]
print("📄 Upload job description (TXT):")
jd_file = list(files.upload().keys())[0]

# Prepare initial state
initial_state = {
    "resume_text": parse_resume(resume_file),
    "job_description": open(jd_file, 'r').read(),
    "candidate_profile": {},
    "job_requirements": {},
    "match_score": 0.0,
    "confidence": 0.0,
    "recommendation": "",
    "requires_human": False,
    "reasoning_summary": "",
    "parsing_error": False
}

# Run LangGraph app
result = app.invoke(initial_state)

# Final output
final_output = {
    "match_score": result["match_score"],
    "recommendation": result["recommendation"],
    "requires_human": result["requires_human"],
    "confidence": result["confidence"],
    "reasoning_summary": result["reasoning_summary"]
}

print(json.dumps(final_output, indent=2))

📤 Upload resume (PDF):


Saving resume_03_ananya_patel.pdf to resume_03_ananya_patel.pdf
📄 Upload job description (TXT):


Saving jd_03_junior_flexible.txt to jd_03_junior_flexible.txt
{
  "match_score": 0.28,
  "recommendation": "Reject",
  "requires_human": false,
  "confidence": 0.85,
  "reasoning_summary": "Match score 0.28. Matched skills: []. Experience: 0y vs 0y required. Strictness: flexible."
}
